In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from erisk.utils import get_spark

bucket = "gs://dsgt-clef-erisk-2024"
spark = get_spark()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 19:43:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/31 19:43:03 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
from google.cloud import storage

# Lookup Bucket

storage_client = storage.Client()

mybucket = storage_client.lookup_bucket('dsgt-clef-erisk-2024')
print(mybucket)

<Bucket: dsgt-clef-erisk-2024>


In [4]:
blobs = mybucket.list_blobs()

eval_list = []

for blob in blobs:
    if 'eval.json' in blob.name:
        print(blob.name)

        eval_list.append(blob.name)

task1/processed/eval/fm_count/v3/eval.json
task1/processed/eval/fm_word2vec/v3/eval.json
task1/processed/eval/logistic_count/v3/eval.json
task1/processed/eval/logistic_word2vec/v3/eval.json
task1/processed/eval/loopy_nb_count/v3/eval.json
task1/processed/eval/nb_count/v3/eval.json
task1/processed/eval/nb_hashing/v3/eval.json
task1/processed/eval_sample/fm_count/v3/eval.json
task1/processed/eval_sample/fm_word2vec/v3/eval.json
task1/processed/eval_sample/logistic_count/v3/eval.json
task1/processed/eval_sample/logistic_word2vec/v3/eval.json
task1/processed/eval_sample/loopy_nb_count/v3/eval.json
task1/processed/eval_sample/nb_count/v3/eval.json
task1/processed/eval_sample/nb_hashing/v3/eval.json


In [28]:
from pyspark.sql import functions as F

eval1 = spark.read.json(f"{bucket}/task1/processed/eval/*/*/eval.json", multiLine=True)
# eval1.show()
# eval1.withColumn("path",F.input_file_name()).show()
resultsdf = eval1.withColumn("path",F.input_file_name()).toPandas()


In [20]:
# Read a singular file from gc buckets
blob = mybucket.blob(f"task1/processed/eval/fm_count/v3/eval.json")

with blob.open("r") as f:
    print(f.read())

In [29]:
resultsdf

,accuracy,f1,mean_accuracy_test,mean_accuracy_train,mean_f1_test,mean_f1_train,time,path
0,"{'test': {'target_1': 0.8527918781725888, 'tar...","{'test': {'target_1': 0.7850358111396982, 'tar...",0.785812,0.779946,0.732174,0.725416,5173.888613,gs://dsgt-clef-erisk-2024/task1/processed/eval...
1,"{'test': {'target_1': 0.8223350253807107, 'tar...","{'test': {'target_1': 0.7696506087128657, 'tar...",0.875174,0.870619,0.831250,0.822320,241.885394,gs://dsgt-clef-erisk-2024/task1/processed/eval...
2,"{'test': {'target_1': 0.8527918781725888, 'tar...","{'test': {'target_1': 0.7850358111396982, 'tar...",0.868086,0.855428,0.815505,0.799129,1274.265832,gs://dsgt-clef-erisk-2024/task1/processed/eval...
3,"{'test': {'target_1': 0.8375634517766497, 'tar...","{'test': {'target_1': 0.7774069607650671, 'tar...",0.881330,0.876923,0.834353,0.825473,269.669995,gs://dsgt-clef-erisk-2024/task1/processed/eval...
4,"{'test': {'target_1': 0.8375634517766497, 'tar...","{'test': {'target_1': 0.7774069607650671, 'tar...",0.879270,0.876125,0.833316,0.825073,562.322454,gs://dsgt-clef-erisk-2024/task1/processed/eval...
5,"{'test': {'target_1': 0.7766497461928934, 'tar...","{'test': {'target_1': 0.7657233673797927, 'tar...",0.830131,0.992224,0.837461,0.992103,355.536902,gs://dsgt-clef-erisk-2024/task1/processed/eval...
6,"{'test': {'target_1': 0.8223350253807107, 'tar...","{'test': {'target_1': 0.7951432085536678, 'tar...",0.877425,0.942504,0.864343,0.936157,367.077385,gs://dsgt-clef-erisk-2024/task1/processed/eval...


In [31]:
resultsdf['path']

0    gs://dsgt-clef-erisk-2024/task1/processed/eval...
1    gs://dsgt-clef-erisk-2024/task1/processed/eval...
2    gs://dsgt-clef-erisk-2024/task1/processed/eval...
3    gs://dsgt-clef-erisk-2024/task1/processed/eval...
4    gs://dsgt-clef-erisk-2024/task1/processed/eval...
5    gs://dsgt-clef-erisk-2024/task1/processed/eval...
6    gs://dsgt-clef-erisk-2024/task1/processed/eval...
Name: path, dtype: object

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf("string")
def to_upper(s: pd.Series) -> pd.Series:
    return s.split('/')